In [1]:
### IMPORTS
import cv2

from world import *
from world_executor import *
from video_util import *
from metadata_util import *
import lens
import point

#import tasm


In /home/yongming/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/yongming/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/yongming/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/yongming/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/yongming/.local/lib/python3.6/site-packages/matplotlib/

In [7]:
### Let's define some attribute for constructing the world first
name = 'traffic_scene' # world name
units = 'metrics'      # world units
video_file = './amber_videos/traffic-scene-short.mp4' #example video file
lens_attrs = {'fov': 120, 
              'cam_origin': (0, 0, 0), 
              'skew_factor': 0}
point_attrs = {'p_id': 'p1', 
               'cam_id': 'cam1', 
               'x': 0,
               'y': 0, 
               'z': 0,
               'time': None, 
               'type':'pos'}
camera_attrs = {'ratio': 0.5}


In [8]:
### First we define a world
new_world = World(name=name, units=units)

In [9]:
### Secondly we construct the camera
fov, res, cam_origin, skew_factor = lens_attrs['fov'], [1280, 720], lens_attrs['cam_origin'], lens_attrs['skew_factor']
cam_lens = lens.PinholeLens(res, cam_origin, fov, skew_factor)

pt_id, cam_id, x, y, z, time, pt_type = point_attrs['p_id'], point_attrs['cam_id'], point_attrs['x'], point_attrs['y'], point_attrs['z'], point_attrs['time'], point_attrs['type']
location = point.Point(pt_id, cam_id, x, y, z, time, pt_type)

ratio = camera_attrs['ratio']

In [10]:
### Ingest the camera to the world
recog_world = new_world.camera(cam_id=cam_id, 
                               location=location, 
                               ratio=ratio, 
                               video_file=video_file, 
                               metadata_identifier=name+"_"+cam_id, 
                               lens=cam_lens)\
                       .recognize(cam_id)

In [11]:
### Call execute on the world to save the real data to the database
recog_world.execute()

Worlds Table created successfully........
New world inserted successfully........
Camera Table created successfully........
New camera inserted successfully.........


/home/yongming/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


AttributeError: 'VideoContext' object has no attribute 'tasm'

In [ ]:
volume = traffic_world.select_intersection_of_interest_or_use_default(cam_id=cam_id)
filtered_world = traffic_world.predicate(lambda obj:obj.object_type == "car").predicate(lambda obj:obj.location in volume, {"volume":volume})
filtered_world = filtered_world.interval([30*300,30*1200])
 
### to get the trajectory and the video over the entire trajectory(amber case)
filtered_ids = filtered_world.selectkey(distinct = True).execute()
print("filtered_ids are", filtered_ids)
print(len(filtered_ids))
if filtered_ids:
    ### Fetch the trajectory of these items
    trajectory = traffic_world.predicate(lambda obj: obj.object_id in id_array, {"id_array":id_array}).get_trajectory(distinct=True).execute()
    print(trajectory[0])
    
    ### Get the videos of these items
    print("fetch traj time is:", traj_time-start)
    entire_video = traffic_world.predicate(lambda obj: obj.object_id in id_array, {"id_array":id_array}).get_video()
    print(entire_video.execute())